# Scratch work for evaluation functions

In [1]:
# import initial dataset and graphs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

import torch
import torch_geometric
from torch_geometric.utils import to_networkx

import networkx as nx

In [2]:
#initial_smiles = pd.read_csv('pesticides_augmented.csv')
initial_graphs = torch.load("./data/pesticides_graphs_augmented_v3.pt", weights_only=False)
output_novel_graphs = torch.load("./data/decoded_new_points.pt", weights_only=False)

In [3]:
initiadsl_graphs = torch.load("./data/pesticides_graphs_v3.pt", weights_only=False)

In [5]:
print(len(initiadsl_graphs))
len(initial_graphs)

1743


2561

In [4]:
G0 = to_networkx(initial_graphs[0], to_undirected=True)
G1 = to_networkx(initial_graphs[1], to_undirected=True)

In [5]:
nx.is_isomorphic(G0, G1, node_match=None, edge_match=None)

False

In [6]:
output_novel_graphs[4].edge_index

array([[0.00000000e+00, 1.00000000e+00, 2.24493518e-01, ...,
        2.86601881e-10, 3.00226732e-10, 3.01795255e-10],
       [1.00000000e+00, 0.00000000e+00, 7.74688959e-01, ...,
        2.89749141e-10, 2.94509694e-10, 3.00139108e-10],
       [2.24493518e-01, 7.74688959e-01, 0.00000000e+00, ...,
        2.98906261e-10, 2.96266373e-10, 2.91439067e-10],
       ...,
       [2.86601881e-10, 2.89749141e-10, 2.98906261e-10, ...,
        0.00000000e+00, 3.07487119e-10, 3.04536119e-10],
       [3.00226732e-10, 2.94509694e-10, 2.96266373e-10, ...,
        3.07487119e-10, 0.00000000e+00, 2.95405311e-10],
       [3.01795255e-10, 3.00139108e-10, 2.91439067e-10, ...,
        3.04536119e-10, 2.95405311e-10, 0.00000000e+00]])

In [7]:
G0 = nx.from_numpy_array(np.round(output_novel_graphs[0].edge_index), create_using = nx.MultiGraph())
G1 = nx.from_numpy_array(np.round(output_novel_graphs[2].edge_index), create_using = nx.MultiGraph())

In [8]:
nx.is_isomorphic(G0, G1, node_match=None, edge_match=None)

False

In [9]:
type(output_novel_graphs[0].edge_index)

numpy.ndarray

In [10]:
def unpad_graphs(generated_graphs):
    
    temp = []
    unconnected = np.zeros(len(generated_graphs))
    valid_graphs = []
    
    for i, G in enumerate(generated_graphs):
        adj_matrix = np.round(G.edge_index)
        dim = adj_matrix.shape[0] # track the dimensions to know how much padding to remove
        row_sum = adj_matrix.sum(axis=0) # get the sums within each row, which is the same as the column sum
        zero_indices = np.where(row_sum == 0)[0] # get all zero indices
        
        # see if zero indices has increments of one
        # this indicates if the graph is unconnected
        for p in range(1, len(zero_indices)):
            if zero_indices[p] != zero_indices[p - 1] + 1:
                unconnected[i] = 1 # if the graph is unconnected, flag it
                
        idx = zero_indices[0] if zero_indices.size > 0 else -1 # get first zero index
        adjusted_matrix = adj_matrix[0:idx, 0:idx] # only take the nonzero matrix portion of valid graphs
        
        G_update = copy.deepcopy(G) # copy to avoid overwriting
        G_update.edge_index = adjusted_matrix
        
        if unconnected[i] == 0:
            valid_graphs.append(G_update) # this only keeps the valid graphs
        
    print(f'Percent of unconnected (invalid) graphs: {100*sum(unconnected)/len(unconnected)}')
    return valid_graphs

In [11]:
cleaned_graphs = unpad_graphs(output_novel_graphs)

Percent of unconnected (invalid) graphs: 20.0


In [12]:
temp = cleaned_graphs.copy()
holder = copy.deepcopy(initial_graphs[0])
convert = to_networkx(holder, to_undirected=True)
convert = nx.adjacency_matrix(convert)
convert = convert.toarray()
holder.edge_index = convert
temp.append(holder)

In [13]:
temp

[Data(x=[41, 3], edge_index=[3, 3], edge_attr=[82, 1], y=0.2222222222222222),
 Data(x=[41, 3], edge_index=[21, 21], edge_attr=[82, 1], y=0.6666666666666666),
 Data(x=[41, 3], edge_index=[12, 12], edge_attr=[82, 1], y=0.8888888888888888),
 Data(x=[41, 3], edge_index=[12, 12], edge_attr=[82, 1], y=1.1111111111111112),
 Data(x=[41, 3], edge_index=[14, 14], edge_attr=[82, 1], y=1.3333333333333333),
 Data(x=[41, 3], edge_index=[21, 21], edge_attr=[82, 1], y=1.5555555555555554),
 Data(x=[41, 3], edge_index=[19, 19], edge_attr=[82, 1], y=1.7777777777777777),
 Data(x=[41, 3], edge_index=[31, 31], edge_attr=[82, 1], y=2.0),
 Data(x=[11, 3], edge_index=[11, 11], edge_attr=[22, 1], y=1.492900013923645)]

In [14]:
def graph_isomorphism_within_set(generated_graphs):
    
    isomorphic_tracker = np.zeros(len(generated_graphs))
    
    for i, new_graph0 in enumerate(generated_graphs):
        
        G0 = nx.from_numpy_array(np.round(new_graph0.edge_index), create_using = nx.MultiGraph())
        
        for p, new_graph1 in enumerate(generated_graphs):
            G1 = nx.from_numpy_array(np.round(new_graph1.edge_index), create_using = nx.MultiGraph())
            if i != p:
                if nx.is_isomorphic(G0, G1, node_match=None, edge_match=None) == True:
                    isomorphic_tracker[i] = p+1 # move these values off of zero
            if i == p:
                continue
    
    n_unique_graphs = 0
    duplicate_tracker = []
    
    for val in isomorphic_tracker:
        if val == 0:
            n_unique_graphs += 1
        if val != 0:
            duplicate_tracker.append(val)

    n_unique_graphs += len(set(duplicate_tracker))/2

    percent_unique = 100*n_unique_graphs/len(generated_graphs)
    
    return print(f'There are {n_unique_graphs} unique graphs, with an overall {percent_unique}% unique graphs generated')

In [17]:
graph_isomorphism_within_set(temp)

There are 7.0 unique graphs, with an overall 77.77777777777777% unique graphs generated


In [18]:
def graph_isomorphism_between_sets(initial_graphs, generated_graphs):
    
    isomorphic_tracker = np.zeros(len(generated_graphs))
    
    for i, new_graph in enumerate(generated_graphs):
        
        
        G_new = nx.from_numpy_array(np.round(new_graph.edge_index), create_using = nx.MultiGraph())
        
        
        for original_graph in initial_graphs:
            G_old = to_networkx(original_graph, to_undirected=True)
            if nx.is_isomorphic(G_new, G_old, node_match=None, edge_match=None) == True:
                isomorphic_tracker[i] = 1
    percent_unique = 100*(len(generated_graphs) - sum(isomorphic_tracker))/len(generated_graphs)
    return print(f'Compared to the initial graphs used for training, generated graphs are {percent_unique}% unique')

In [19]:
graph_isomorphism_between_sets(initial_graphs[0:2], temp)

Compared to the initial graphs used for training, generated graphs are 88.88888888888889% unique


In [20]:
temp[7]

Data(x=[41, 3], edge_index=[31, 31], edge_attr=[82, 1], y=2.0)

In [21]:
import rdkit
from rdkit.Chem.QED import properties
from rdkit import Chem

from functions.data_generation import *

In [22]:
G_test= nx.from_numpy_array(np.round(temp[7].edge_index), create_using = nx.MultiGraph())

In [23]:
test = nx.write_adjlist(G_test, "test.adjlist")
#mol = to_smiles(temp[7])

In [24]:
test

In [25]:
test = nx.read_adjlist("test.adjlist")

In [26]:
test

In [27]:
test_mol = copy.deepcopy(temp[7])
edges_converted = test.edges
test_mol.edge_index = torch.swapaxes(torch.tensor(np.array(edges_converted, dtype=int)), 0, 1)

In [28]:
print(test_mol)
print(test_mol.x)
print(test_mol.x[0, 0].item())
print(test_mol.edge_index)
print(test_mol.edge_attr)

Data(x=[41, 3], edge_index=[2, 34], edge_attr=[82, 1], y=2.0)
[[ 4.5933337   4.617169    5.119407  ]
 [ 7.7466874   2.7827387   5.1082387 ]
 [ 9.205264    1.9964241   5.021751  ]
 [ 6.6676526   2.46109     5.0309677 ]
 [ 6.7248383   1.4075874   4.8812933 ]
 [ 4.074717    3.4154994   4.7670727 ]
 [ 4.0530953   3.722745    4.746638  ]
 [ 5.941204    1.2560456   4.6296186 ]
 [ 4.1850314   3.5944939   4.5404034 ]
 [ 6.4733157   3.700335    4.747698  ]
 [ 9.355043    4.090858    5.599221  ]
 [11.830396    3.3624198   5.3138185 ]
 [ 7.3152413   2.1579413   5.553982  ]
 [10.984373    2.586135    5.494074  ]
 [ 8.295892    3.2678695   5.6614347 ]
 [ 7.2209      2.204265    5.4892235 ]
 [10.22921     1.9652436   5.8147345 ]
 [ 7.390416    4.5736456   6.2776055 ]
 [ 7.1431465   3.3978858   6.6462464 ]
 [ 8.310295    2.9066803   6.7853966 ]
 [ 7.6558      4.0037994   6.339445  ]
 [ 8.795314    3.740126    6.6360164 ]
 [ 9.023699    4.6168876   6.6542926 ]
 [ 9.226395    3.6263943   6.6896706 ]
 [

In [29]:
print(initial_graphs[0])
print(initial_graphs[0].x)
print(initial_graphs[0].edge_index)
print(initial_graphs[0].edge_attr)

Data(x=[11, 3], edge_index=[2, 22], edge_attr=[22, 1], y=1.492900013923645)
tensor([[ 6.,  4.,  5.],
        [ 6.,  4.,  5.],
        [ 6.,  4.,  5.],
        [ 6.,  3.,  5.],
        [ 8.,  1.,  5.],
        [ 7.,  3.,  5.],
        [17.,  1.,  5.],
        [ 6.,  3.,  5.],
        [ 8.,  1.,  5.],
        [ 7.,  3.,  5.],
        [35.,  1.,  5.]])
tensor([[ 0,  1,  1,  1,  1,  2,  3,  3,  3,  4,  5,  5,  5,  6,  7,  7,  7,  8,
          9,  9,  9, 10],
        [ 1,  0,  2,  3,  9,  1,  1,  4,  5,  3,  3,  6,  7,  5,  5,  8,  9,  7,
          1,  7, 10,  9]])
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [2.],
        [1.],
        [2.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [2.],
        [1.],
        [2.],
        [1.],
        [1.],
        [1.],
        [1.]])


In [30]:
convert_test = test_mol

In [31]:
mol = Chem.RWMol()

for i in range(convert_test.num_nodes):
    atom = Chem.Atom(int(convert_test.x[i, 0].item()))
    atom.SetFormalCharge(x_map['formal_charge'][int(convert_test.x[i, 2].item())])
    mol.AddAtom(atom)

edges = [tuple(i) for i in convert_test.edge_index.t().tolist()]
visited = set()

for i in range(len(edges)):
    src, dst = edges[i]
    #if tuple(sorted(edges[i])) in visited:
    #    continue

    bond_type = Chem.BondType.values[int(convert_test.edge_attr[i, 0].item())]
    mol.AddBond(src, dst, bond_type)

    #visited.add(tuple(sorted(edges[i])))

mol = mol.GetMol()

Chem.SanitizeMol(mol)
#Chem.AssignStereochemistry(mol)

Chem.MolToSmiles(mol, isomericSmiles=True)

[21:11:16] Explicit valence for atom # 4 C, 7, is greater than permitted


AtomValenceException: Explicit valence for atom # 4 C, 7, is greater than permitted

In [ ]:
edges

In [40]:
output_graphs = torch.load("./data/decoded_new_points.pth", weights_only=False)

FileNotFoundError: [Errno 2] No such file or directory: './data/decoded_new_points.pth'

In [39]:
valid_graphs = unpad_graphs(output_graphs)

NameError: name 'output_graphs' is not defined

In [38]:
graph_isomorphism_within_set(valid_graphs)

NameError: name 'valid_graphs' is not defined

In [33]:
def mol_reconstruct(valid_graphs):
    '''
    This function takes valid graphs with adjacency matrices at their edge index position
    and tries to reconstruct a valid molecule
    '''
    
    mol_out = []
    valid_mol = []
    validity_tracker = 0
    
    # first reconstruct the node features, edge indices, and edge features in appropriate format
    for graph in valid_graphs:
        graph_temp = copy.deepcopy(graph) 

        D = graph_temp.edge_index.shape[0] # determine dimension of adjacency matrix
        
        # this block takes the adjacency matrix and reconstructs edge indices needed for RDKIT
        G_temp= nx.from_numpy_array(graph_temp.edge_index, create_using = nx.MultiGraph())
        nx.write_adjlist(G_temp, "gtemp.adjlist")
        G_edge_pairs = nx.read_adjlist("gtemp.adjlist")
        
        edge_pair_inverse = G_edge_pairs.edges
        graph_temp.edge_index = torch.swapaxes(torch.tensor(np.array(edge_pair_inverse, dtype=int)), 0, 1)
        D_edge = graph_temp.edge_index.shape[1]

        # this block takes the node features and rounds them needed for RDKIT
        graph_temp.x = np.array(np.round(graph_temp.x), dtype=int)[0:D]

        # this block takes the node features and rounds them needed for RDKIT
        graph_temp.edge_attr = np.array(np.round(graph_temp.edge_attr), dtype=int)[0:D_edge]

        # now attempt to reconstruct valid molecules
        mol = Chem.RWMol()

        for i in range(graph_temp.num_nodes):
            atom = Chem.Atom(int(graph_temp.x[i, 0].item()))
            atom.SetFormalCharge(x_map['formal_charge'][int(graph_temp.x[i, 2].item())])
            mol.AddAtom(atom)
        
        edges = [tuple(i) for i in graph_temp.edge_index.t().tolist()]
        
        for i in range(len(edges)):
            src, dst = edges[i]
        
            bond_type = Chem.BondType.values[int(graph_temp.edge_attr[i, 0].item())]
            mol.AddBond(src, dst, bond_type)
        
        reconstructed_mol = mol.GetMol()
        mol_out.append(reconstructed_mol)

        try:
            Chem.SanitizeMol(mol)
            is_sanitized = True
            valid_mol.append(reconstructed_mol)
            validity_tracker += 1
            
        except Chem.AtomValenceException as e:
            is_sanitized = False            
            
    print(f'There are {validity_tracker} valid molecules, with a percent validity of {100*(validity_tracker/len(valid_graphs))}%')
    
    return mol_out, valid_mol

In [34]:
all_out, valid_mol = mol_reconstruct(valid_graphs)

NameError: name 'valid_graphs' is not defined

In [ ]:
valid_mol[0]

In [ ]:
example_graph = nx.from_numpy_array(np.round(valid_graphs[0].edge_index), create_using = nx.MultiGraph())

plt.figure(1,figsize=(7,6)) 
nx.draw(example_graph, node_size=5,linewidths=6)
plt.show()


In [42]:
initial_graphs = torch.load("./data/pesticides_graphs_augmented_v3.pt", weights_only=False)

In [43]:
initial_graphs

[Data(x=[11, 3], edge_index=[2, 22], edge_attr=[22, 1], y=1.492900013923645),
 Data(x=[10, 3], edge_index=[2, 20], edge_attr=[20, 1], y=2.248199939727783),
 Data(x=[10, 3], edge_index=[2, 20], edge_attr=[20, 1], y=2.248199939727783),
 Data(x=[10, 3], edge_index=[2, 20], edge_attr=[20, 1], y=2.248199939727783),
 Data(x=[25, 3], edge_index=[2, 50], edge_attr=[50, 1], y=6.077899932861328),
 Data(x=[11, 3], edge_index=[2, 22], edge_attr=[22, 1], y=-0.7333999872207642),
 Data(x=[13, 3], edge_index=[2, 28], edge_attr=[28, 1], y=2.2929000854492188),
 Data(x=[11, 3], edge_index=[2, 26], edge_attr=[26, 1], y=-0.8751999735832214),
 Data(x=[9, 3], edge_index=[2, 16], edge_attr=[16, 1], y=2.339200019836426),
 Data(x=[10, 3], edge_index=[2, 22], edge_attr=[22, 1], y=1.5895999670028687),
 Data(x=[14, 3], edge_index=[2, 30], edge_attr=[30, 1], y=2.863100051879883),
 Data(x=[10, 3], edge_index=[2, 18], edge_attr=[18, 1], y=2.342360019683838),
 Data(x=[5, 3], edge_index=[2, 8], edge_attr=[8, 1], y=1.85